In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

## Overfitting / Gradient Vanishing / Gradient Exploding

### 과적합(Overfitting)과 막는 방법

`과적합(Overfitting)` 은 학습 데이터를 모델이 지나치게 학습하여 검증 데이터에 대해 오히려 성능이 떨어지는 현상

모델의 성능을 떨어트리는 주요 이슈, 훈련 데이터에 대한 정확도는 높을지라도, 새로운 데이터, 즉 검증 데이터나 테스트 데이터에 대해서는 제대로 동작하지 않음

모델이 학습 데이터를 불필요할정도로 과하게 암기하여 훈련 데이터에 포함된 노이즈까지 학습한 상태로 해석할 수 있음

### 데이터 증강(Data Augmentation)

모델은 데이터의 양이 적을 경우, 해당 데이터의 특정 패턴이나 노이즈까지 쉽게 암기하기 되므로 과적합 현상이 발생할 확률이 증가

따라서 데이터의 양을 늘릴 수록 모델은 데이터의 일반적인 패턴을 학습하여 과적합을 방지

`데이터 증강(Data Augmentation)`은 데이터의 양이 적을 경우 의도적으로 기존의 데이터를 조금씩 변형하고 추가하여 데이터의 양을 늘리는 방법

데이터 증강에서 가장 중요한건 데이터에서 핵심적인 부분을 보존하면서 변형해야 한다는 것

이미지 데이터의 데이터 증강 예시
- 회전, 이동, 크기 조정, 뒤집기, 자르기, 색상 조정, 노이즈 주입

텍스트 데이터의 데이터 증강 예시
- 동의어 대체, 역번역(텍스트를 다른 언어로 번역 후 원래 언어로 다시 번역하는 것), 무작위 단어 삽입/삭제, 의역(Paraphrasing)

오디오 데이터의 데이터 증강 예시
- 시간늘리기, 피치 이동, 동적 범위 변형, 노이즈 주입

### 모델의 복잡도 줄이기

인공 신경망의 복잡도는 은닉층(hidden layer)의 수나 매개변수의 수 등으로 결정

과적합 현상이 포착되었을 때, 인공 신경망 모델에 대해서 할 수 있는 한 가지 조치는 인공 신경망의 복잡도를 줄이는 것

인공 신경망에서는 모델에 있는 매개변수들의 수를 모델의 수용력(capacity)이라고 하기도 함

In [24]:
# 3개의 선형 층
class Architecture1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Architecture1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [25]:
# 2개의 선형 층
class Architecture1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Architecture1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

### 가중치 규제(Regularization) 적용

`가중치 규제(Regularization)` 는 손실 함수에 페널티를 추가하여 과적합을 방지하는 방법

- L1 regularization : 가중치 w들의 절대값 합계를 비용 함수에 추가, L1 norm 라고도 함
- L2 regularization : 모든 가중치 w들의 제곱합을 비용 함수에 추가, L2 norm 라고도 함

두 식 모두 비용 함수를 최소화하기 위해서는 가중치 w들의 값이 작아져야 한다는 특징이 있음

가중치 w의 값들은 0 또는 0에 가까이 작아져야 하므로 어떤 특성들은 모델을 만들 때 거의 사용되지 않게 됨

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = Net()

`L2 규제`는 `가중치 감쇠(weight decay)`라고도 부르며, 옵티마이저의 매개변수 `weight_decay`를 통해 사용가능하며 기본값은 0이다

In [27]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# loop
criterion = nn.MSELoss()
data, targets = torch.randn(10, 10), torch.randn(10, 1)
optimizer.zero_grad()
outputs = model(data)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

`L1 규제`는 PyTorch 옵티마이저 매개변수를 통해 직접 지원되지 않지만 손실을 수정하여 수동으로 추가할 수 있다

In [28]:
def l1_regularization(model, lambda_l1):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return lambda_l1 * l1_norm

# loop
optimizer.zero_grad()
outputs = model(data)
mse_loss = criterion(outputs, targets)
l1_loss = l1_regularization(model, lambda_l1=0.01)
total_loss = mse_loss + l1_loss
total_loss.backward()
optimizer.step()

### 드롭아웃(Dropout)

`드롭아웃(Dropout)`은 학습 과정에서 신경망의 일부를 사용하지 않는 방법

드롭아웃의 비율을 0.5로 한다면 학습 과정마다 랜덤으로 절반의 뉴런을 사용하지 않고, 절반의 뉴런만을 사용

드롭아웃은 신경망 학습 시에만 사용하고, 예측 시에는 사용하지 않는 것이 일반적

학습 시에 인공 신경망이 특정 뉴런 또는 특정 조합에 너무 의존적이게 되는 것을 방지

매번 랜덤 선택으로 뉴런들을 사용하지 않으므로 서로 다른 신경망들을 앙상블하여 사용하는 것 같은 효과를 내어 과적합을 방지

In [29]:
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super(FullyConnectedNet, self).__init__()
        self.fc1 = nn.Linear(20, 50)
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 비율이 0.5
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## 기울기 소실(Gradient Vanishing)과 폭주(Exploding)

`기울기 소실(Gradient Vanishing)` : 역전파 과정에서 입력층으로 갈 수록 기울기(Gradient)가 점차적으로 작아지는 현상, 이 경우 입력층에 가까운 층들에서 가중치들이 업데이트가 제대로 되지 않으면 결국 최적의 모델을 찾을 수 없게 됨

`기울기 폭주(Gradient Exploding)` : 기울기 소실(Gradient Vanishing)의 반대의 현상, 기울기가 점차 커지더니 가중치들이 비정상적으로 큰 값이 되면서 결국 발산, 순환 신경망(Recurrent Neural Network, RNN)에서 발생할 수 있음

### 가중치 초기화(Weight initialization)

같은 모델을 훈련시키더라도 가중치가 초기에 어떤 값을 가졌느냐에 따라서 모델의 훈련 결과가 달라지기도 함

### 세이비어 초기화(Xavier Initialization) / 글로럿 초기화(Glorot Initialization)

세이비어 초기화는 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막음

`sigmoid` 함수나 `tanh` 함수와 같은 S자 형태인 활성화 함수와 함께 사용할 경우에는 `좋은 성능`

ReLU와 함께 사용할 경우에는 성능이 좋지 않음

세이비어 초기화는 균등 분포(Uniform Distribution) 또는 정규 분포(Normal distribution)로 초기화 할 때 두 가지 경우로 나뉘며

이전 층의 뉴런 개수 $n_{in}$, 다음 층의 뉴런 개수 $n_{out}$ 이라 할때 다음과 같은 균등 분포 범위를 사용함

$ W \sim Uniform(- \sqrt{\frac{6}{n_{in} + n_{out}}}, + \sqrt{\frac{6}{n_{in} + n_{out}}}) $

정규 분포로 초기화할 경우에는 평균이 0이고, 표준 편차 $\sigma$가 다음을 만족하도록 함

$ \sigma = \sqrt{\frac{2}{n_{in} + n_{out}}} $

### He 초기화(He initialization)

He 초기화는 `ReLU` 계열 함수를 사용한 경우 세이비어 초기화 보다 `좋은 성능`을 보임

He 초기화는 세이비어 초기화와 유사하게 정규 분포와 균등 분포 두 가지 경우로 나뉨

세이비어 초기화와 달리 다음 층의 뉴런의 수를 반영하지 않음, 이전 층의 뉴런 개수 $n_{in}$라 할때 다음과 같은 균등 분포 범위를 가짐

$ W \sim Uniform(- \sqrt{\frac{6}{n_{in}}}, + \sqrt{\frac{6}{n_{in}}}) $

정규 분포로 초기화할 경우에는 평균이 0이고, 표준 편차 $\sigma$가 다음을 만족하도록 함

$ \sigma = \sqrt{\frac{2}{n_{in}}} $

### 배치 정규화(Batch Normalization)

`배치 정규화(Batch Normalization)`는 인공 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만듬

이전 층들의 학습에 의해 이전 층의 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력 데이터의 분포가 현재 층이 학습했던 시점의 분포와 차이가 발생

즉 학습 과정에서 층 별로 입력 데이터 분포가 달라지는 현상이 발생하고 이를 `내부 공변량 변화(Internal Covariate Shift)`라 함

배치 정규화는 표현 그대로 한 번에 들어오는 배치 단위로 정규화하는 것을 뜻함

학습 시 배치 단위의 평균과 분산들을 차례대로 받아 이동 평균과 이동 분산을 저장해놓았다가 

테스트 할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화함

배치 단위가 아닌 층 단위 정규화를 `층 정규화(Layer Normalization)`라 함

배치 정규화를 사용하면 시그모이드 함수나 하이퍼볼릭탄젠트 함수를 사용하더라도 기울기 소실 문제가 크게 개선

가중치 초기화에 훨씬 덜 민감

훨씬 큰 학습률을 사용할 수 있어 학습 속도를 개선

미니 배치마다 평균과 표준편차를 계산하므로 훈련 데이터에 일종의 잡음을 넣는 부수 효과로 과적합을 방지하는 효과, 다만  부수적 효과이므로 드롭 아웃과 함께 사용하는 것이 좋음

배치 정규화는 모델을 복잡하게 하며, 추가 계산을 하는 것이므로 테스트 데이터에 대한 예측 시에 실행 시간이 느려짐, 서비스 속도를 고려하는 관점에서는 배치 정규화가 꼭 필요한지 고민 필요

배치 정규화는 너무 작은 배치 크기에서는 잘 동작하지 않을 수 있음, 배치 정규화를 적용할때는 작은 미니 배치보다는 크기가 어느정도 되는 미니 배치에서 하는 것이 좋음

RNN은 각 시점(time step)마다 다른 통계치를 가지기에 배치 정규화를 적용하는 것을 어렵게 함, RNN의 경우 층 정규화 방법을 사용